In [2]:
import re
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import json
import pandas as pd
import csv
from tqdm import tqdm

In [139]:
def init_driver(start_url):
    service = Service(executable_path="chromedriver_win.exe")

    driver = webdriver.Chrome(service=service)

    driver.implicitly_wait(5)

    driver.get(start_url)

    driver.maximize_window()

    top_div = driver.find_element(By.ID, 'usercentrics-root')
    shadow_root = top_div.shadow_root
    confirm_button = shadow_root.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")

    confirm_button.click()



In [ ]:
def scrap_commune(driver, zip_code, com_name, property_type):
    url_list = []
    ulist = driver.find_element(By.CLASS_NAME, "pagination")
    litems = ulist.find_elements(By.TAG_NAME, "li")
    pages_number = int(litems[-2].find_elements(By.TAG_NAME, "span")[1].text)

    for i in range(pages_number):        
        soup = bs(driver.page_source, 'html.parser')
        main_list = soup.find('ul', id='main-content')
        links = main_list.find_all('a')
        for link in links:
            url = link['href']
            if "immoweb.be" in url:
                url_list.append(url)        
        
        if i < pages_number - 1:                        
            litems[-1].click()
            time.sleep(2.5)

    url_book = {'zip_code': zip_code, 'com_name': com_name, 'urls': url_list}

    url_book_json = json.dumps(url_book, indent=4)
    with open(f'url_store/{com_name}_{property_type}.json', 'w') as save_file:
        save_file.write(url_book_json)


    return url_list

In [206]:
def change_commune(driver, zip_to_go):
    form = driver.find_element(By.CLASS_NAME, "multiselect__form")

    inputter = form.find_element(By.TAG_NAME, "input")

    remove_button = form.find_element(By.TAG_NAME, "button")

    remove_button.click()

    inputter.send_keys(zip_to_go)
    time.sleep(0.3)
    inputter.send_keys(Keys.ARROW_DOWN)
    time.sleep(0.3)
    inputter.send_keys(Keys.ENTER)


In [ ]:
def bot_session(starting_point):
    service = Service(executable_path="chromedriver_win.exe")

    driver = webdriver.Chrome(service=service)

    driver.implicitly_wait(5)

    driver.get(starting_point)

    driver.maximize_window()

    top_div = driver.find_element(By.ID, 'usercentrics-root')
    shadow_root = top_div.shadow_root
    confirm_button = shadow_root.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")

    confirm_button.click()

    url_list = []
    ulist = driver.find_element(By.CLASS_NAME, "pagination")
    litems = ulist.find_elements(By.TAG_NAME, "li")
    pages_number = int(litems[-2].find_elements(By.TAG_NAME, "span")[1].text)
    

    for i in range(165):        
        soup = bs(driver.page_source, 'html.parser')
        main_list = soup.find('ul', id='main-content')
        links = main_list.find_all('a')
        for link in links:
            url = link['href']
            if "immoweb.be" in url:
                url_list.append(url)        
        
        if i < pages_number - 1:                        
            litems[-1].click()
            time.sleep(2)

    url_book = {'urls': url_list}

    url_book_json = json.dumps(url_book, indent=4)
    with open(f'url_store/houses_all_2.json', 'w') as save_file:
        save_file.write(url_book_json)

    

In [30]:
df = pd.read_json('url_store/appartments_all.json')
df_2 = pd.read_json('url_store/appartments_all_2.json')

df_joined = pd.concat([df, df_2], ignore_index=True)
print(df_joined.shape)
df_joined.drop_duplicates(inplace=True)
print(df_joined.shape)

df_joined.to_csv('url_store/apartments_links.csv', index=False)

(9931, 1)
(9619, 1)


In [33]:
df = pd.read_json('url_store/houses_all.json')
df_2 = pd.read_json('url_store/houses_all_2.json')

df_joined = pd.concat([df, df_2], ignore_index=True)
print(df_joined.shape)
df_joined.drop_duplicates(inplace=True)
print(df_joined.shape)

df_joined.to_csv('url_store/houses_links.csv', index=False)

(9931, 1)
(9587, 1)


In [3]:
def immo_page_scrapper(source_html, property_type=0):
    real_estate = {'locality': None, 'zip_code': None, 'type_of_property': property_type, 'subtype_of_property': None, 'Price': None, 'building_condition': None, 'facade_number': None, 'living_area': None, 
               'equipped_kitchen': 0, 'bedroom_nr': None, 'swimming_pool': 0, 'furnished': 0, 'open_fire': 0, 'Terrace': 0, 'garden': 0, 'plot_surface': None}
    additional_links = []

    soup = bs(source_html, 'html.parser')    

    flag_text = soup.find('span', class_="flag-list__text").text

    #excluding life annuity

    if flag_text == "Life annuity":
        return False

    type_in_title = soup.find('h1', class_="classified__title").text.replace("for sale", "")
    real_estate['subtype_of_property'] = type_in_title.strip()

    address_rows = soup.find_all('span', class_="classified__information--address-row")

    for add_row in address_rows:        
        if re.search("\d{4}", add_row.text):
            locality_text = add_row.text.strip()
            real_estate['zip_code'] = int(re.findall("(^\d{4})", locality_text)[0])
            real_estate['locality'] = (re.findall("([^0-9 ]+$)", locality_text)[0])

    second_headers = soup.find_all('h2')

    for second_h in second_headers:
        #treatment of bundeled offerings, will return list of internal links
        if second_h.string == "All properties":                       
            internal_links = second_h.parent.parent.find_all('a')
            for internal_link in internal_links:                
                additional_links.append(internal_link['href'])
            return additional_links

        elif second_h.string == "Description":            
            paragraphs = second_h.parent.find_all('p')
            for par in paragraphs:
                if "open fire" in par.text:
                    real_estate['open_fire'] = 1

        elif second_h.string == "General":            
            rows = second_h.parent.parent.find_all('tr')
            for row in rows:                
                for cell in row.children:
                    if "Building condition" in cell.text:                        
                        real_estate['building_condition'] = (cell.find_next_sibling('td').text.strip())
                    elif "Number of frontages" in cell.text:                        
                        real_estate['facade_number'] = int(cell.find_next_sibling('td').text.strip())

        elif second_h.string == "Interior":
            rows = second_h.parent.parent.find_all('tr')
            for row in rows:
                for cell in row.children:
                    if "Living area" in cell.text:
                        liv_area = re.findall("(\d+)", cell.find_next_sibling('td').text)[0]
                        real_estate['living_area'] = int(liv_area)
                    elif "Kitchen type" in cell.text:
                        real_estate['equipped_kitchen'] = cell.find_next_sibling('td').text.strip()
                    elif "Bedrooms" in cell.text:
                        real_estate['bedroom_nr'] = int(cell.find_next_sibling('td').text.strip())
                    elif "Furnished" in cell.text and "Yes" in cell.find_next_sibling('td').text:
                        real_estate['Furnished'] = 1

        elif second_h.string == "Exterior":
            rows = second_h.parent.parent.find_all('tr')
            for row in rows:
                for cell in row.children:
                    if "Surface of the plot" in cell.text:
                        plot_area = re.findall("(\d+)", cell.find_next_sibling('td').text)[0]
                        real_estate['plot_surface'] = plot_area
                    elif "Garden surface" in cell.text:
                        garden_area = re.findall("(\d+)", cell.find_next_sibling('td').text)[0]
                        real_estate['garden'] = garden_area
                    elif "Terrace surface" in cell.text:
                        terrace_area = re.findall("(\d+)", cell.find_next_sibling('td').text)[0]
                        real_estate['Terrace'] = terrace_area

        elif second_h.string == "Facilities":
            rows = second_h.parent.parent.find_all('tr')
            for row in rows:
                for cell in row.children:
                    if "Swimming pool" in cell.text and "Yes" in cell.find_next_sibling('td').text:
                        real_estate['swimming_pool'] = 1
                    

        elif second_h.string == "Financial":
            wrap_div = second_h.parent.parent
            rows = wrap_div.find_all('tr')
            for row in rows:
                for cell in row.children:
                    if "Price" in cell.text:
                        expr = re.compile("\d+\s€")
                        price_strings = re.findall(expr, cell.find_next_sibling('td').text)[0]
                        price_int = int(price_strings[:-2])
                        real_estate['Price'] = price_int

        
    #if no Life annuity or offering bundle returns dictionary with extracted info
    return real_estate

In [4]:
def requests_scrapper(html_text, property_type=0):
    real_estate = {'immo_id': None, 'zip_code': None, 'type_of_property': property_type, 'subtype_of_property': None, 'price': None, 'building_condition': None, 'facade_number': None, 'living_area': None, 
               'equipped_kitchen': 0, 'bedroom_nr': None, 'swimming_pool': 0, 'furnished': 0, 'open_fire': 0, 'terrace': 0, 'garden': 0, 'plot_surface': 0}
    url_list = []

    soup = bs(html_text, 'html.parser')    

    info_script = soup.find('script', string=re.compile("window.dataLayer.push"))
    expr = re.compile('"classified":[\S\n ]+"customer":')   
    dict_like = re.findall(expr, info_script.string)[0]
    dict_like = re.sub('"classified":', '', dict_like)
    dict_like = re.sub(',\n\s*"customer":', '', dict_like)        
    prop_dict = (json.loads(dict_like))    

    annuitant_check = soup.find_all('th', string=re.compile("annuitant"))
    title = soup.find('title').text

    if len(annuitant_check) > 0:
        return False

    elif len(re.findall("(\s\d+m)", title)) > 0:
        real_estate['living_area'] = int(re.findall("(\s\d+m)", title)[0][1:-1])

        frontages_header = soup.find("th", class_="classified-table__header", string=re.compile("Number of frontages"))
        if frontages_header is not None:
                real_estate['facade_number'] = int(frontages_header.parent.find('td').string.strip())

        terrace_header = soup.find("th", class_="classified-table__header", string=re.compile("Terrace surface"))
        if terrace_header is not None:
            data_cell = terrace_header.parent.find('td')
            real_estate['terrace'] = int(re.findall("(\d+)", str(data_cell))[0])

        furnished_header = soup.find("th", class_="classified-table__header", string=re.compile("Furnished"))
        if furnished_header is not None:
            if furnished_header.parent.find('td').string.strip() == "Yes":
                real_estate['furnished'] = 1

        descr = soup.find('p', class_="classified__description")
        if descr is not None:
            if 'open fire' in descr.text:
                real_estate['open_fire'] = 1

        real_estate['zip_code'] = int(prop_dict['zip'])
        real_estate['immo_id'] = prop_dict['id']
        real_estate['subtype_of_property'] = prop_dict['subtype']
        if prop_dict['price'] == "no price":
            real_estate['price'] = None
        else:    
            real_estate['price'] = int(prop_dict['price'])
        real_estate['building_condition'] = prop_dict['building']['condition']    
        real_estate['bedroom_nr'] = int(prop_dict['bedroom']['count'])
        if prop_dict['kitchen']['type'] != "":
            real_estate['equipped_kitchen'] = prop_dict['kitchen']['type']
        if prop_dict['wellnessEquipment']['hasSwimmingPool'] == "true":
            real_estate['swimming_pool'] = 1
        if prop_dict['outdoor']['terrace']['exists'] == "true" and real_estate['terrace'] == 0:
            real_estate['terrace'] = 1
        if 'surface' in prop_dict['outdoor']['garden'] and prop_dict['outdoor']['garden']['surface'] != "":
            real_estate['garden'] = int(prop_dict['outdoor']['garden']['surface'])
        if prop_dict['land']['surface'] != "":
            real_estate['plot_surface'] = int(prop_dict['land']['surface'])

        return real_estate
    
    elif "group" in prop_dict['type']:
        anchor = soup.find('template', string=re.compile("All properties"))
        for link in anchor.parent.find_all('a'):
            url_list.append([link['href']])
        
        return url_list
    
    else:
        return True

In [6]:
url_complete = []
url_good = []
with open('Alek_scrapper_results/houses_additional_urls.csv', newline='') as f:
    reader = csv.reader(f)    
    for thing in reader:
        url_complete.append(thing[0])



print(url_complete)

['https://www.immoweb.be/en/classified/house/for-sale/noorderwijk/2200/20316264', 'https://www.immoweb.be/en/classified/house/for-sale/noorderwijk/2200/20316267', 'https://www.immoweb.be/en/classified/house/for-sale/noorderwijk/2200/20316270', 'https://www.immoweb.be/en/classified/house/for-sale/noorderwijk/2200/20316265', 'https://www.immoweb.be/en/classified/house/for-sale/noorderwijk/2200/20316268', 'https://www.immoweb.be/en/classified/house/for-sale/noorderwijk/2200/20316269', 'https://www.immoweb.be/en/classified/house/for-sale/noorderwijk/2200/20316262', 'https://www.immoweb.be/en/classified/house/for-sale/noorderwijk/2200/20316263', 'https://www.immoweb.be/en/classified/house/for-sale/noorderwijk/2200/20316266', 'https://www.immoweb.be/en/classified/villa/for-sale/zulte-machelen/9870/20311323', 'https://www.immoweb.be/en/classified/house/for-sale/gent/9000/20314337', 'https://www.immoweb.be/en/classified/house/for-sale/gent/9000/20314302', 'https://www.immoweb.be/en/classified/

In [7]:
field_names = ['immo_id', 'zip_code', 'type_of_property', 'subtype_of_property', 'price', 'building_condition', 'facade_number', 'living_area', 'equipped_kitchen', 'bedroom_nr', 'swimming_pool', 'furnished', 'open_fire', 'terrace', 'garden', 'plot_surface']

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

session = requests.Session()

session.headers = headers

#with open('apartment_data.csv', 'a', newline='') as file:
#    csv.writer(file).writerow(field_names)

for link in tqdm(url_complete):
    response = session.get(link, headers=headers)
    try:
        result = requests_scrapper(response.text)
        if isinstance(result, dict):
            with open('Alek_scrapper_results/houses_data.csv', 'a', newline='') as file:
                csv.DictWriter(file, fieldnames=field_names).writerow(result)
        elif isinstance(result, list):
            with open('Alek_scrapper_results/houses_additional_urls.csv', 'a', newline='') as file:
                for url in result:
                    csv.writer(file).writerow(url)
    except Exception as ex:
        with open('Alek_scrapper_results/houses_failed_urls.csv', 'a', newline='') as file:
            csv.writer(file).writerow([link])        
        continue

  0%|          | 0/1973 [00:00<?, ?it/s]

100%|██████████| 1973/1973 [11:16<00:00,  2.92it/s]


In [157]:
with open('Alek_scrapper_results/houses_data.csv', 'a', newline='') as file:
    csv.writer(file).writerow(field_names)

In [129]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

real_estate = {'zip_code': None, 'type_of_property': 0, 'subtype_of_property': None, 'price': None, 'building_condition': None, 'facade_number': None, 'living_area': None, 
               'equipped_kitchen': 0, 'bedroom_nr': None, 'swimming_pool': 0, 'furnished': 0, 'open_fire': 0, 'terrace': 0, 'garden': 0, 'plot_surface': 0}

url_list = []

session = requests.Session()

session.headers = headers

response = session.get(url_complete[0])

soup = bs(response.text, 'html.parser')

print(soup)

info_script = soup.find('script', string=re.compile("window.dataLayer.push"))
expr = re.compile('"classified":[\S\n ]+"customer":')   
dict_like = re.findall(expr, info_script.string)[0]
dict_like = re.sub('"classified":', '', dict_like)
dict_like = re.sub(',\n\s*"customer":', '', dict_like)        
prop_dict = (json.loads(dict_like))
print(dict_like)

title = soup.find('title').text

if len(re.findall("(\s\d+m)", title)) > 0:
    annuitant_check = soup.find_all('th', string=re.compile("annuitant"))

    real_estate['living_area'] = int(re.findall("(\s\d+m)", title)[0][1:-1])    

    frontages_header = soup.find("th", class_="classified-table__header", string=re.compile("Number of frontages"))
    if frontages_header is not None:
        real_estate['facade_number'] = int(frontages_header.parent.find('td').string.strip())

    terrace_header = soup.find("th", class_="classified-table__header", string=re.compile("Terrace surface"))
    if terrace_header is not None:
        data_cell = terrace_header.parent.find('td')
        real_estate['terrace'] = int(re.findall("(\d+)", str(data_cell))[0])

    furnished_header = soup.find("th", class_="classified-table__header", string=re.compile("Furnished"))
    if furnished_header is not None:
        if furnished_header.parent.find('td').string.strip() == "Yes":
            real_estate['furnished'] = 1

    descr = soup.find('p', class_="classified__description")
    if descr is not None:
        if 'open fire' in descr.text:
            real_estate['open_fire'] = 1

    real_estate['zip_code'] = int(prop_dict['zip'])
    real_estate['subtype_of_property'] = prop_dict['subtype']
    if prop_dict['price'] == "no price":
        real_estate['price'] = None
    else:    
        real_estate['price'] = int(prop_dict['price'])
    real_estate['building_condition'] = prop_dict['building']['condition']    
    real_estate['bedroom_nr'] = int(prop_dict['bedroom']['count'])
    if prop_dict['kitchen']['type'] != "":
        real_estate['equipped_kitchen'] = prop_dict['kitchen']['type']
    if prop_dict['wellnessEquipment']['hasSwimmingPool'] == "true":
        real_estate['swimming_pool'] = 1
    if prop_dict['outdoor']['terrace']['exists'] == "true" and real_estate['terrace'] == 0:
        real_estate['terrace'] = 1
    if 'surface' in prop_dict['outdoor']['garden'] and prop_dict['outdoor']['garden']['surface'] != "":
        real_estate['garden'] = int(prop_dict['outdoor']['garden']['surface'])
    if prop_dict['land']['surface'] != "":
        real_estate['plot_surface'] = int(prop_dict['land']['surface'])

    print(real_estate)

elif "group" in prop_dict['type']:
    anchor = soup.find('template', string=re.compile("All properties"))
    for link in anchor.parent.find_all('a'):
        url_list.append(link['href'])
    
    print(url_list)

else:
    print('something else')



InvalidSchema: No connection adapters were found for "['https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/anderlecht/1070/20317200']"

In [57]:
service = Service(executable_path="chromedriver_win.exe")

driver = webdriver.Chrome(service=service)

driver.implicitly_wait(5)

driver.get("https://www.immoweb.be/en")

driver.maximize_window()

top_div = driver.find_element(By.ID, 'usercentrics-root')
shadow_root = top_div.shadow_root
confirm_button = shadow_root.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")

confirm_button.click()

cookies = driver.get_cookies()

session = requests.Session()

for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

session.headers = headers

response = session.get('https://www.immoweb.be/en/classified/apartment/for-sale/berchem/2600/20316946')

soup = bs(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="en">
 <head>
  <meta charset="utf-8"/>
  <script>
   if (!Array.prototype.flat) { window.location.replace('https://www.immoweb.be/en/outdated-browser') }
  </script>
  <script type="text/javascript">
   const variantName = "AI_in_Advanced_Search";
    const isABTestWithCookie = false;

    /* START: AB Test Homepage - AI tab for unauthenticated users. */
    const LDConsent = hasCookieConsentById("vagdYWLSXoC06v");
    /* END: AB Test Homepage - AI tab for unauthenticated users. */

    if (
        /* START: AB Test Homepage - AI tab for unauthenticated users. */
        !LDConsent ||
        /* END: AB Test Homepage - AI tab for unauthenticated users. */
        !variantName || isABTestWithCookie) { // AB test is not configured or started using cookie for storing variant
        localStorage.removeItem("ab-test");
        window.ABTestVariant = null;
    } else {
        setABTestVariant();
    }

    funct

In [ ]:
"""
Locality + +
Type of property (House/apartment) - binary is House + +
Subtype of property (Bungalow, Chalet, Mansion, ...) + +
Price + +
Type of sale (Exclusion of life sales) +
Number of rooms +
Living Area + 
Fully equipped kitchen (Yes/No) + 
Furnished (Yes/No) +
Open fire (Yes/No) +
Terrace (Yes/No) 
If yes: Area +
Garden (Yes/No) +
If yes: Area +
Surface of the land (+)
Surface area of the plot of land +
Number of facades +
Swimming pool (Yes/No) +
State of the building (New, to be renovated, ...) +
"""

'\nLocality +\nType of property (House/apartment) - binary is House +\nSubtype of property (Bungalow, Chalet, Mansion, ...) +\nPrice +\nType of sale (Exclusion of life sales) +\nNumber of rooms +\nLiving Area + \nFully equipped kitchen (Yes/No) + \nFurnished (Yes/No) +\nOpen fire (Yes/No) +\nTerrace (Yes/No) \nIf yes: Area +\nGarden (Yes/No) +\nIf yes: Area +\nSurface of the land (+)\nSurface area of the plot of land +\nNumber of facades +\nSwimming pool (Yes/No) +\nState of the building (New, to be renovated, ...) +\n'